In [ ]:
import sys
import time
import mariadb

In [ ]:
# Connect Mariadb
try:
    conn = mariadb.connect(
        user="root",
        password="123456",
#         host="127.0.0.1",
#         port=3306,
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

In [ ]:
DB_NAME = 'wse'

TABLES = {}
TABLES['lexicon'] = (
    "CREATE TABLE `lexicon` ("
    "  `term`   varchar(30) NOT NULL,"
    "  `start`  bigint      NOT NULL,"
    "  `end`    bigint      NOT NULL,"
    "  `freq`   int         NOT NULL,"
    "  PRIMARY KEY (`term`)"
    ") ENGINE=InnoDB")

TABLES['docs'] = (
    "CREATE TABLE `docs` ("
    "  `url`   varchar(1024)   NOT NULL,"
    "  `text`  varchar(2048)   NOT NULL,"
    "  PRIMARY KEY (`url`)"
    ") ENGINE=InnoDB")

TABLES['pgurl'] = (
    "CREATE TABLE `pgurl` ("
    "  `id`    int             NOT NULL,"
    "  `url`   varchar(1024)   NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

In [ ]:
# Get Cursor
cur = conn.cursor()

In [ ]:
# Create Database
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cur.execute("USE {}".format(DB_NAME))
except conn.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    create_database(cur)
    print("Database {} created successfully.".format(DB_NAME))
    conn.database = DB_NAME

In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table: {} ".format(table_name))
        cur.execute(table_description)
    except conn.Error as err:
        print(err.msg)

In [ ]:
# build lexicon
start_time = time.time()
with open('/Volumes/Elements/indexer/lexicon.txt') as lex:
    for line in lex:
        term, start, end, freq = line.strip().split()
        try:
            cur.execute(
                "INSERT INTO lexicon (term, start, end, freq) VALUES (?, ?, ?, ?)", 
                (term, int(start), int(end), int(freq)))
        except conn.Error as err:
            print(err)
conn.commit()
print(time.time() - start_time)

# 1099s

In [ ]:
# build url table
start_time = time.time()
did = 0
with open('/Volumes/Elements/indexer/page_url.txt') as pg:
    for line in pg:
        url = line.strip()
        try:
            cur.execute(
                "INSERT INTO pgurl (id, url) VALUES (?, ?)", 
                (int(did), url))
        except conn.Error as err:
            print(err)
        did += 1
conn.commit()
print(time.time() - start_time)

# 104s

In [ ]:
# build page content table
start_time = time.time()
with open('/Volumes/Elements/indexer/msmarco-docs.trec') as d:
    while True:
        nxt = d.readline()
        if len(nxt) == 0:
            break
        if nxt.strip() == "<DOC>":
            for i in range(2):
                d.readline()
            url = d.readline().strip()
            text = ''
            line = ''
            while True:
                line = d.readline().strip()
                if line == "</TEXT>":
                    break
                text += line + ' '
            try:
                cur.execute(
                    "INSERT INTO docs (url, text) VALUES (?, ?)", 
                    (url, text[:2048]))
            except conn.Error as err:
                pass
conn.commit()
print(time.time() - start_time)

# 2032s

In [ ]:
conn.close()